<a href="https://colab.research.google.com/drive/1nCh7YtUZ4vqCxkdpV5PrqhfZIxvowMr_?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Accelerating Dask with GPUs (via RAPIDS)

We've seen in lecture how the RAPIDS libraries make it possible to accelerate common analytical workflows on GPUs using libraries like cudf (for GPU DataFrames) and cuml (for basic GPU machine learning operations on DataFrames). When your data gets especially large (e.g. exceeding the memory capacity of a single GPU) or your computations get especially cumbersome, Dask makes it possible to scale these workflows out even further -- distributing work out across a cluster of GPUs.

In AWS Academy, recall that we cannot create GPU clusters. However, this notebook should also be runnable on multi-GPU EC2 instances and clusters (on AWS) if you use a personal account to request these resources. Here (on Colab), we'll demonstrate using a single GPU. Note that the setup portion of this notebook draws on [a setup notebook](https://colab.research.google.com/drive/13sspqiEZwso4NYTbsflpPyNFaVAAxUgr) linked in the RAPIDS documentation and is meant to be run in a Colab notebook.

This demo is built off of the notebooks provided in the [RAPIDS notebook repositories](https://github.com/rapidsai/notebooks) on GitHub (and you can explore them further if you are interested! There are many other relevant libraries in the RAPIDS ecosystem -- e.g. `cugraph` which allows you to perform network analyses on GPUs).

## Setup

Click the _Runtime_ dropdown at the top of the page, then _Change Runtime Type_ and confirm the instance type is _GPU_.

Check the output of `!nvidia-smi` to make sure you've been allocated a Tesla T4, P4, or P100.

In [1]:
!nvidia-smi

Sun May  7 17:49:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Then we run the setup script below, which:

1. Checks to make sure that the GPU is RAPIDS compatible
1. Installs the **current stable version** of RAPIDSAI's core libraries using pip and **will complete in about 3-4 minutes**

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

At this point, our RAPIDS libraries are now installed on Colab and we can import them into our session. Let's use `dask_cuda`'s API to launch a Dask GPU cluster and pass this cluster object to our `dask.distributed` client. `LocalCUDACluster()` will count each available GPU in our cluster (in this case, 1 GPU) as a Dask worker and assign it work.

In [ ]:
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from dask_cuda import LocalCUDACluster
from dask.distributed import Client

cluster = LocalCUDACluster() # Identify all available GPUs
client = Client(cluster)

## GPU DataFrames

From here, we can use `dask_cudf` to automate the process of partitioning our data across our GPU workers and instantiating a GPU-based DataFrame on our GPU that we can work with. Let's load in the same AirBnB data that we were working with in the `numba` + `dask` CPU demonstration:

In [4]:
import dask_cudf

df = dask_cudf.read_csv('listings*.csv')
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,3781,HARBORSIDE-Walk to subway,4804,Frank,<NA>,East Boston,42.36413,-71.02991,Entire home/apt,125,32,19,2021-02-26,0.27,1,106
1,6695,$99 Special!! Home Away! Condo,8229,Terry,<NA>,Roxbury,42.32802,-71.09387,Entire home/apt,169,29,115,2019-11-02,0.81,4,40
2,10813,"Back Bay Apt-blocks to subway, Newbury St, The...",38997,Michelle,<NA>,Back Bay,42.35061,-71.08787,Entire home/apt,96,29,5,2020-12-02,0.08,11,307
3,10986,North End (Waterfront area) CLOSE TO MGH & SU...,38997,Michelle,<NA>,North End,42.36377,-71.05206,Entire home/apt,96,29,2,2016-05-23,0.03,11,293
4,13247,Back Bay studio apartment,51637,Susan,<NA>,Back Bay,42.35164,-71.08752,Entire home/apt,75,91,0,<NA>,<NA>,2,0


Once we have that data, we can perform many of the standard DataFrame operations we perform on CPUs -- just accelerated by our GPU cluster!

In [5]:
df.groupby(['neighbourhood', 'room_type']) \
  .price \
  .mean() \
  .compute()

neighbourhood   room_type      
North Center    Private room        75.818182
Clearing        Entire home/apt     90.000000
Edgewater       Entire home/apt    140.142857
South Lawndale  Entire home/apt     79.826087
Auburn Gresham  Entire home/apt    135.000000
                                      ...    
Lakeshore       Entire home/apt    205.500000
Brighton Park   Shared room         39.000000
Lake View       Hotel room         656.400000
North Beach     Shared room         31.900000
Ashburn         Entire home/apt    100.857143
Name: price, Length: 341, dtype: float64

One thing to note, though, is that not all of the functionality we might expect out of CPU clusters is available yet in the `cudf`/`dask_cudf` DataFrame implementation.

For instance (and of particular note!), our ability to apply custom functions is still pretty limited. `cudf` uses Numba's CUDA compiler to translate this code for the GPU and [many standard `numpy` operations are not supported](https://numba.pydata.org/numba-doc/dev/cuda/cudapysupported.html#numpy-support) (for instance, if you try to apply the distance calculation with performed in the Numba+Dask CPU demonstration notebook for today, this will fail to compile correctly for the GPU).

That being said, we can perform many base-Python operations inside of custom functions, so if you can express your custom functions in this way, it might be worth your while to do this work on a GPU. For example, let's create a custom price index that indicates whether an AirBnB is \"Cheap\" (0), \"Moderately Expensive\" (1), or \"Very Expensive\" (2) using `cudf`'s [`apply_rows` method](https://docs.rapids.ai/api/cudf/stable/user_guide/guide-to-udfs.html#numba-kernels-for-dataframes):

In [6]:
def expensive(x, price_index):
    # passed through Numba's CUDA compiler and `for`
    # loop is automatically parallelized for GPU
    for i, price in enumerate(x):
        if price < 50:
            price_index[i] = 0
        elif price < 100:
            price_index[i] = 1
        else:
            price_index[i] = 2

# Use cudf's `apply_rows` API for applying function to every row in DataFrame
df = df.apply_rows(expensive,
                   incols={'price':'x'},
                   outcols={'price_index': int},
                   kwargs={})

# Confirm that price index created correctly
df[['price', 'price_index']].head()

,price,price_index
0,125,2
1,169,2
2,96,1
3,96,1
4,75,1


## Training Machine Learning Models with `cuml`

In addition to preprocessing and analyzing data on GPUs, we can also train (a limited set of) Machine Learning models directly on our GPU cluster using the `cuml` library in the RAPIDS ecoystem as well. This can give us a significant speedup in training time over libraries like `sklearn` on CPUs for large datasets.

For instance, let's train a linear regression model based on our data from San Francisco, Chicago, and Boston to predict the price of an AirBnB based on other values in its listing information (e.g. \"reviews per month\" and \"minimum nights\"). We'll then use this model to make predictions about the price of AirBnBs in another city (NYC):

In [ ]:
from cuml.dask.linear_model import LinearRegression
import numpy as np

X = df[['reviews_per_month', 'minimum_nights']].astype(np.float32).dropna()
y = df[['price']].astype(np.float32).dropna()
fit = LinearRegression().fit(X, y)

Then, we can read in the NYC dataset and make predictions about what prices will be in NYC on the basis of the model we trained on data from our three original cities:

In [9]:
df_nyc = dask_cudf.read_csv('test*.csv')
X_test = df_nyc[['reviews_per_month', 'minimum_nights']].astype(np.float32) \
                                                        .dropna()
fit.predict(X_test) \
   .compute() \
   .head()

0    184.802887
1    188.286636
2    184.802887
3    183.658218
4    186.646774
dtype: float32

If we take a look at other standard machine learning algorithms in the documentation (for instance [k means clustering](https://github.com/rapidsai/cuml/blob/branch-23.04/notebooks/kmeans_demo.ipynb)) as well, we can see significant speedups over performing the same operations on large datasets in scikit-learn on a CPU.

Note, though, that this is only true of larger data. For smaller data sizes, we will see comparable performance on CPU and GPUs. **Ask:** why?